In [2]:
# Importando as Bibliotecas:

import pandas as pd
import plotly.express as px
from haversine import haversine as hs
import plotly.graph_objects as go
import streamlit as st
from PIL import Image
import numpy as np
import folium
import datetime
from streamlit_folium import folium_static
import inflection

# --------------------------------------
# Importando o data set:
df_raw = pd.read_csv("zomato.csv")
df = df_raw.copy()

In [209]:
df.head(3)

,Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,...,Currency,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes
0,6310675,Mama Lou's Italian Kitchen,162,Las Piñas City,"Block 1, Lot 36, Tropical Avenue Corner Tropic...",BF International,"BF International, Las Piñas City",121.009787,14.447615,Italian,...,Botswana Pula(P),1,0,0,0,3,4.6,3F7E00,Excellent,619
1,6310675,Mama Lou's Italian Kitchen,162,Las Piñas City,"Block 1, Lot 36, Tropical Avenue Corner Tropic...",BF International,"BF International, Las Piñas City",121.009787,14.447615,Italian,...,Botswana Pula(P),1,0,0,0,3,4.6,3F7E00,Excellent,619
2,6314542,Blackbird,162,Makati City,"Nielson Tower, Ayala Triangle Gardens, Salcedo...","Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...",121.024562,14.556042,"European, Asian",...,Botswana Pula(P),0,0,0,0,4,4.7,3F7E00,Excellent,469


In [210]:
print(df.shape)

(7527, 21)


## Limpeza do Conjunto de Dados e Funções


In [3]:
# 1. Retirar a coluna de valores vazios 'Swith to order menu':
df = df.drop("Switch to order menu", axis=1)

# 2. Renomeando as colunas do Dataframe:
df = df.rename(columns={"Restaurant ID": "Restaurant_ID", "Restaurant Name": "Restaurant_Name", "Country Code": "Country_Name", "Locality Verbose": "Locality_Verbose", "Average Cost for two": "Average_Cost_for_two", "Has Table booking": "Has_Table_booking", "Has Online delivery": "Has_Online_delivery", "Is delivering now": "Is_delivering_now", "Price range": "Price_range", "Aggregate rating": "Aggregate_rating", "Rating color": "Rating_color", "Rating text": "Rating_text"})
# Sugerido pelo professor:
#def rename_columns(dataframe):
#  df = dataframe.copy()
#  title = lambda x: inflection.titleize(x)
#  snakecase = lambda x: inflection.underscore(x)
#  spaces = lambda x: x.replace(" ", "")
#  cols_old = list(df.columns)
#  cols_old = list(map(title, cols_old))
#  cols_old = list(map(spaces, cols_old))
#  cols_new = list(map(snakecase, cols_old))
#  df.columns = cols_new
#  return df

# 3. Para colocar o nome dos países com base no código de cada país:
COUNTRIES = {
1: "India",
14: "Australia",
30: "Brazil",
37: "Canada",
94: "Indonesia",
148: "New Zeland",
162: "Philippines",
166: "Qatar",
184: "Singapure",
189: "South Africa",
191: "Sri Lanka",
208: "Turkey",
214: "United Arab Emirates",
215: "England",
216: "United States of America",
}
def country_name(Country_ID):
  return COUNTRIES[Country_ID]

df["Country_Name"] = df["Country_Name"].map(country_name)

# 4. Criação do nome das Cores:
COLORS = {
"3F7E00": "darkgreen",
"5BA829": "green",
"9ACD32": "lightgreen",
"CDD614": "orange",
"FFBA00": "red",
"CBCBC8": "darkred",
"FF7800": "darkred",
}
def color_name(color_code):
  return COLORS[color_code]

df["Rating_color"] = df["Rating_color"].map(color_name)

# 5. Criação do Tipo de Categoria de Comida
def create_price_tye(Price_range):
  if Price_range == 1:
    return "cheap"
  elif Price_range == 2:
    return "normal"
  elif Price_range == 3:
    return "expensive"
  else:
    return "gourmet"

df["Price_range"] = df["Price_range"].map(create_price_tye)

# 6. Verificando se tem Dados Duplicados:
df_sem_duplicatas = df.drop_duplicates()
df = df_sem_duplicatas 

# 7. Remove os NA que forem np.na:
df = df.dropna()
# Verificar se há valores nulos ou NaN em cada coluna
#colunas_com_nulos = df.isnull().any()
# Exibir as colunas com valores nulos ou NaN
#print(colunas_com_nulos)
# Contar o número de valores nulos ou NaN em cada coluna
#contagem_nulos = df.isnull().sum()
# Exibir a contagem de nulos por coluna
#print(contagem_nulos)

# 8. Categorizar todos os restaurantes somente por um tipo de culinária:
df["Cuisines"] = df.loc[:, "Cuisines"].astype(str).apply(lambda x: x.split(",")[0])

# 9. Retirar Registros com Média Negativa de Avaliação:
linhas_negativas = df["Cuisines"] != "Mineira"
df = df.loc[linhas_negativas, :]
linhas_negativas2 = df["Cuisines"] != "Drinks Only"
df = df.loc[linhas_negativas2,:]

C:\Users\corte\AppData\Local\Temp\ipykernel_16044\181282775.py:86: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Cuisines"] = df.loc[:, "Cuisines"].astype(str).apply(lambda x: x.split(",")[0])


In [4]:

df.shape

(6927, 20)

## Resolução das Perguntas Propostas

In [249]:
df.columns

Index(['Restaurant_ID', 'Restaurant_Name', 'Country_Name', 'City', 'Address',
       'Locality', 'Locality_Verbose', 'Longitude', 'Latitude', 'Cuisines',
       'Average_Cost_for_two', 'Currency', 'Has_Table_booking',
       'Has_Online_delivery', 'Is_delivering_now', 'Price_range',
       'Aggregate_rating', 'Rating_color', 'Rating_text', 'Votes'],
      dtype='object')

In [7]:
df.head()

,Restaurant_ID,Restaurant_Name,Country_Name,City,Address,Locality,Locality_Verbose,Longitude,Latitude,Cuisines,Average_Cost_for_two,Currency,Has_Table_booking,Has_Online_delivery,Is_delivering_now,Price_range,Aggregate_rating,Rating_color,Rating_text,Votes
0,6310675,Mama Lou's Italian Kitchen,Philippines,Las Piñas City,"Block 1, Lot 36, Tropical Avenue Corner Tropic...",BF International,"BF International, Las Piñas City",121.009787,14.447615,Italian,1100,Botswana Pula(P),1,0,0,expensive,4.6,darkgreen,Excellent,619
2,6314542,Blackbird,Philippines,Makati City,"Nielson Tower, Ayala Triangle Gardens, Salcedo...","Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...",121.024562,14.556042,European,3100,Botswana Pula(P),0,0,0,gourmet,4.7,darkgreen,Excellent,469
3,6301293,Banapple,Philippines,Makati City,"Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...",121.023171,14.556196,Filipino,800,Botswana Pula(P),0,0,0,expensive,4.4,green,Very Good,867
4,6315689,Bad Bird,Philippines,Makati City,"Hole In The Wall, Floor 4, Century City Mall, ...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027708,14.565899,American,700,Botswana Pula(P),0,0,0,expensive,4.4,green,Very Good,858
5,6304833,Manam,Philippines,Makati City,"Level 1, Greenbelt 2, Ayala Center, Greenbelt,...","Greenbelt 2, San Lorenzo, Makati City","Greenbelt 2, San Lorenzo, Makati City, Makati ...",121.020380,14.552351,Filipino,700,Botswana Pula(P),0,0,0,expensive,4.7,darkgreen,Excellent,930


## Geral

#### 1. Quantos restaurantes únicos estão registrados?

In [251]:
restaurantes_unicos = df["Restaurant_ID"].nunique()
print(f"Estão registrados {restaurantes_unicos} restaurantes na plataforma.")

Estão registrados 6927 restaurantes na plataforma.


#### 2. Quantos países únicos estão registrados?

In [6]:
paises_unicos = df["Country_Name"].nunique()
print(f"Estão registrados {paises_unicos} países na plataforma.")

Estão registrados 15 países na plataforma.


#### 3. Quantas cidades únicas estão registradas?

In [206]:
cidades_unicas = df["City"].nunique()
print(f"Estão registradas {cidades_unicas} cidades na plataforma.")

Estão registradas 125 cidades na plataforma.


#### 4. Qual o total de avaliações feitas?

In [207]:
total_avaliacoes = df["Votes"].sum()
print(f"Foram feitas {total_avaliacoes} avaliações na plataforma.")

Foram feitas 4194530 avaliações na plataforma.


#### 5. Qual o total de tipos de culinária registrados?

In [208]:
tipos_culinaria = df["Cuisines"].nunique()
print(f"Foram registradas {tipos_culinaria} tipos de culinária na plataforma.")

Foram registradas 163 tipos de culinária na plataforma.


## Países

#### 1. Qual o nome do país que possui mais cidades registradas?

In [209]:
paises_com_mais_cidades = df.loc[:, ["Country_Name", "City"]].groupby(["Country_Name"]).nunique().sort_values(["City"], ascending=False).reset_index()
paises_com_mais_cidades 
pais_com_mais_cidades = paises_com_mais_cidades.iloc[0:1, 0:2]
print(pais_com_mais_cidades)
valor_pais_com_mais_cidades = paises_com_mais_cidades.iloc[0,1]
print("")
print(f"O país é a Índia com {valor_pais_com_mais_cidades} cidades registradas.")



  Country_Name  City
0        India    49

O país é a Índia com 49 cidades registradas.


#### 2. Qual o nome do país que possui mais restaurantes registrados?

In [11]:
paises_com_mais_restaurantes = df.loc[:, ["Country_Name", "Restaurant_ID"]].groupby("Country_Name").count().sort_values(["Restaurant_ID"], ascending=False).reset_index()
paises_com_mais_restaurantes
pais_com_mais_restaurantes = paises_com_mais_restaurantes.iloc[0:1, 0:2]
print(pais_com_mais_restaurantes)
valor_pais_com_mais_restaurantes = paises_com_mais_restaurantes.iloc[0,1]
print("")
print(f"O país é a Índia com {valor_pais_com_mais_restaurantes} restaurantes registradas.")


  Country_Name  Restaurant_ID
0        India           3111

O país é a Índia com 3111 restaurantes registradas.


#### 3. Qual o nome do país que possui mais restaurantes com o nível de preço igual a 4 registrados?

In [210]:
paises_quatro = df.loc[(df["Aggregate_rating"] == 4), ["Country_Name", "Aggregate_rating"]]
df_paises_quatro = paises_quatro.loc[:, ["Country_Name", "Aggregate_rating"]].groupby("Country_Name").count().sort_values(["Aggregate_rating"], ascending=False).reset_index()
print(df_paises_quatro.iloc[0:1, 0:2])
valor_df_paises_quatro = df_paises_quatro.iloc[0,1]
print("")
print(f"O país é a Índia com {valor_df_paises_quatro} restaurantes registrados nessas condições.")

  Country_Name  Aggregate_rating
0        India               348

O país é a Índia com 348 restaurantes registrados nessas condições.


#### 4. Qual o nome do país que possui a maior quantidade de tipos de culinária distintos?

In [211]:
df_pais_culinaria_distinta = df.loc[:, ["Country_Name","Cuisines"]].groupby("Country_Name").nunique().sort_values(["Cuisines"], ascending=False).reset_index()
print(df_pais_culinaria_distinta.iloc[0:1, 0:2])
valor_df_pais_culinaria_distinta = df_pais_culinaria_distinta.iloc[0,1]
print("")
print(f"O país é a Índia com {valor_df_pais_culinaria_distinta} tipos de culinárias.")

  Country_Name  Cuisines
0        India        77

O país é a Índia com 77 tipos de culinárias.


#### 5. Qual o nome do país que possui a maior quantidade de avaliações feitas?

In [212]:
df_pais_mais_avaliacoes = df.loc[:, ["Country_Name", "Votes"]].groupby("Country_Name").sum().sort_values(["Votes"], ascending=False).reset_index()
print(df_pais_mais_avaliacoes.iloc[0:1, 0:2])
valor_df_pais_mais_avaliacoes = df_pais_mais_avaliacoes.iloc[0,1]
print("")
print(f"O país é a Índia com {valor_df_pais_mais_avaliacoes} avaliações feitas.")

  Country_Name    Votes
0        India  2800164

O país é a Índia com 2800164 avaliações feitas.


#### 6. Qual o nome do país que possui a maior quantidade de restaurantes que fazem entrega?

In [213]:
paises_com_entrega = df.loc[(df["Has_Online_delivery"] == 1), ["Country_Name", "Is_delivering_now"]]
df_paises_com_entrega = paises_com_entrega.loc[:,["Country_Name", "Is_delivering_now"]].groupby("Country_Name").count().sort_values(["Is_delivering_now"], ascending=False).reset_index()
print(df_paises_com_entrega.iloc[0:1,0:2])
valor_df_paises_com_entrega = df_paises_com_entrega.iloc[0,1]
print("")
print(f"O país é a Índia com {valor_df_paises_com_entrega} restaurantes que fazem entrega.")


  Country_Name  Is_delivering_now
0        India               2177

O país é a Índia com 2177 restaurantes que fazem entrega.


#### 7. Qual o nome do país que possui a maior quantidade de restaurantes que aceitam reservas?

In [214]:
paises_com_reserva = df.loc[(df["Has_Table_booking"] == 1), ["Country_Name", "Has_Table_booking"]]
df_paises_com_reserva = paises_com_reserva.loc[:,["Country_Name", "Has_Table_booking"]].groupby("Country_Name").count().sort_values(["Has_Table_booking"], ascending=False).reset_index()
print(df_paises_com_reserva.iloc[0:1,0:2])
valor_df_paises_com_reserva = df_paises_com_reserva.iloc[0,1]
print("")
print(f"O país é a Índia com {valor_df_paises_com_reserva} restaurantes que aceitam reservas de mesas.")

  Country_Name  Has_Table_booking
0        India                256

O país é a Índia com 256 restaurantes que aceitam reservas de mesas.


#### 8. Qual o nome do país que possui, na média, a maior quantidade de avaliações registrada?

In [215]:
df_pais_mais_avaliacoes = df.loc[:, ["Country_Name", "Votes"]].groupby("Country_Name").sum().sort_values(["Votes"], ascending=False).reset_index()
#print(df_pais_mais_avaliacoes)
paises_com_mais_restaurantes = df.loc[:, ["Country_Name", "Restaurant_ID"]].groupby("Country_Name").count().sort_values(["Restaurant_ID"], ascending=False).reset_index()
#print(paises_com_mais_restaurantes)
df_aux = pd.merge(df_pais_mais_avaliacoes, paises_com_mais_restaurantes, how="inner")
df_aux["paises_media"] = df_aux["Votes"] / df_aux["Restaurant_ID"]
valor_df_aux = df_aux.iloc[0, 3]
valor_df_aux = round(valor_df_aux,2)
print("")
print(f"O país é a Índia com {valor_df_aux} avaliações de média registrada.")


O país é a Índia com 900.08 avaliações de média registrada.


#### 9. Qual o nome do país que possui, na média, a maior nota média registrada?

In [216]:
df_pais_medias = df.loc[:,["Country_Name", "Aggregate_rating"]].groupby("Country_Name").mean().sort_values(["Aggregate_rating"], ascending=False).reset_index()
valor_df_pais_medias= round(df_pais_medias.iloc[0,1],2)
valor_df_pais_medias
print(f"O país é a Indonesia com maior nota média registrada de {valor_df_pais_medias}.")


O país é a Indonesia com maior nota média registrada de 4.6.


#### 10. Qual o nome do país que possui, na média, a menor nota média registrada?

In [217]:
df_pais_medias_min = df.loc[:,["Country_Name", "Aggregate_rating"]].groupby("Country_Name").mean().sort_values(["Aggregate_rating"], ascending=True).reset_index()
valor_df_pais_medias_min= round(df_pais_medias_min.iloc[0,1],2)
valor_df_pais_medias_min
print(f"O país é o Brasil com a menor nota média registrada de {valor_df_pais_medias_min}.")


O país é o Brasil com a menor nota média registrada de 3.34.


#### 11. Qual a média de preço de um prato para dois por país?

In [218]:
media_preco_prato_pais = df.loc[:, ["Country_Name", "Average_Cost_for_two"]].groupby("Country_Name").mean().sort_values(["Average_Cost_for_two"], ascending=False).reset_index()
media_preco_prato_pais

,Country_Name,Average_Cost_for_two
0,Indonesia,303000.000000
1,Australia,138959.783333
2,Sri Lanka,2579.375000
3,Philippines,1227.825000
4,India,704.400514
5,South Africa,339.515942
6,Qatar,174.000000
7,United Arab Emirates,153.716667
8,Singapure,141.437500
9,Brazil,139.393305


## Cidade

#### 1. Qual o nome da cidade que possui mais restaurantes registrados?

In [219]:
df_cidades_mais_restaurantes = df.loc[:,["Restaurant_ID", "City"]].groupby("City").count().sort_values(["Restaurant_ID"], ascending=False).reset_index()
print(df_cidades_mais_restaurantes.head(5))
valor_df_cidades_mais_restaurantes = df_cidades_mais_restaurantes.iloc[0,1]
print("")
print(f"A cidade com mais restaurante é Abu Dhabi com {valor_df_cidades_mais_restaurantes} restaurantes registrados")


        City  Restaurant_ID
0  Abu Dhabi             80
1  Mangalore             80
2  Edinburgh             80
3    Glasgow             80
4        Goa             80

A cidade com mais restaurante é Abu Dhabi com 80 restaurantes registrados


#### 2. Qual o nome da cidade que possui mais restaurantes com nota média acima de 4?

In [220]:
cidades_mais_restaurantes_nota4 = df.loc[(df["Aggregate_rating"] >= 4), ["City", "Aggregate_rating"]]
df_cidades_mais_restaurantes_nota4 = cidades_mais_restaurantes_nota4.loc[:,["City", "Aggregate_rating"]].groupby("City").count().sort_values(["Aggregate_rating"], ascending=False).reset_index()
print(df_cidades_mais_restaurantes_nota4.head(3))
valor_df_cidades_mais_restaurantes_nota4 = df_cidades_mais_restaurantes_nota4.iloc[0,1]
print("")
print(f"A cidade que possui mais restaurantes nessas condições é Londres com {valor_df_cidades_mais_restaurantes_nota4} restaurantes.")

        City  Aggregate_rating
0     London                80
1  Bangalore                79
2    Houston                77

A cidade que possui mais restaurantes nessas condições é Londres com 80 restaurantes.


#### 3. Qual o nome da cidade que possui mais restaurantes com nota média abaixo de 2.5?

In [221]:
cidades_mais_restaurantes_nota25 = df.loc[(df["Aggregate_rating"] <= 2.5), ["City", "Aggregate_rating"]]
df_cidades_mais_restaurantes_nota25 = cidades_mais_restaurantes_nota25.loc[:,["City", "Aggregate_rating"]].groupby("City").count().sort_values(["Aggregate_rating"], ascending=False).reset_index()
print(df_cidades_mais_restaurantes_nota25.head(3))
valor_df_cidades_mais_restaurantes_nota25 = df_cidades_mais_restaurantes_nota25.iloc[0,1]
print("")
print(f"A cidade que possui mais restaurantes nessas condições é Gangtok com {valor_df_cidades_mais_restaurantes_nota25} restaurantes.")

       City  Aggregate_rating
0   Gangtok                33
1      Ooty                19
2  Brasília                16

A cidade que possui mais restaurantes nessas condições é Gangtok com 33 restaurantes.


#### 4. Qual o nome da cidade que possui o maior valor médio de um prato para dois?

In [24]:
media_preco_prato_cidade = df.loc[:, ["City", "Average_Cost_for_two"]].groupby("City").mean().sort_values(["Average_Cost_for_two"], ascending=False).reset_index()
print(media_preco_prato_cidade.head(3))
valor_media_preco_prato_cidade = round(media_preco_prato_cidade.iloc[0,1],2)
print("")
print(f"A cidade que mais possui restaurantes nessas condições é Adelaide com uma média de {valor_media_preco_prato_cidade}.")

        City  Average_Cost_for_two
0   Adelaide         416734.133333
1    Jakarta         309605.263158
2  Tangerang         180000.000000

A cidade que mais possui restaurantes nessas condições é Adelaide com uma média de 416734.13.


#### 5. Qual o nome da cidade que possui a maior quantidade de tipos de culinária distintas?

In [222]:
cidades_maior_tipos_culinaria = df.loc[:, ["City", "Cuisines"]].groupby("City").nunique().sort_values(["Cuisines"], ascending=False).reset_index()
print(cidades_maior_tipos_culinaria.head(3))
valor_cidades_maior_tipos_culinaria = cidades_maior_tipos_culinaria.iloc[0,1]
print("")
print(f"A cidade que possui a maior variedade de culinárias é Birmingham com {valor_cidades_maior_tipos_culinaria} culinárias.")

         City  Cuisines
0  Birmingham        32
1        Doha        31
2    Montreal        30

A cidade que possui a maior variedade de culinárias é Birmingham com 32 culinárias.


#### 6. Qual o nome da cidade que possui a maior quantidade de restaurantes que fazem reservas?

In [223]:
cidades_com_reserva = df.loc[(df["Has_Table_booking"] == 1), ["City", "Has_Table_booking"]]
df_cidades_com_reserva = cidades_com_reserva.loc[:,["City", "Has_Table_booking"]].groupby("City").count().sort_values(["Has_Table_booking"], ascending=False).reset_index()
print(df_cidades_com_reserva.head(3))
valor_df_cidades_com_reserva = df_cidades_com_reserva.iloc[0,1]
print("")
print(f" A cidade que possui mais restaurantes nessas condições é Bangalore com {valor_df_cidades_com_reserva} restaurantes.")

        City  Has_Table_booking
0  Bangalore                 42
1    Chennai                 28
2       Pune                 25

 A cidade que possui mais restaurantes nessas condições é Bangalore com 42 restaurantes.


#### 7. Qual o nome da cidade que possui a maior quantidade de restaurantes que fazem entregas?

In [224]:
cidades_com_entrega = df.loc[(df["Is_delivering_now"] == 1), ["City", "Is_delivering_now"]]
df_cidades_com_entrega = cidades_com_entrega.loc[:,["City", "Is_delivering_now"]].groupby("City").count().sort_values(["Is_delivering_now"], ascending=False).reset_index()
print(df_cidades_com_entrega.head(3))
valor_df_cidades_com_entrega = df_cidades_com_entrega.iloc[0,1]
print("")
print(f" A cidade que possui mais restaurantes nessas condições é Vadodara com {valor_df_cidades_com_entrega} restaurantes.")

         City  Is_delivering_now
0    Vadodara                 48
1    Amritsar                 48
2  Aurangabad                 47

 A cidade que possui mais restaurantes nessas condições é Vadodara com 48 restaurantes.


#### 8. Qual o nome da cidade que possui a maior quantidade de restaurantes que aceitam pedidos online?

In [225]:
cidades_com_entrega_online = df.loc[(df["Has_Online_delivery"] == 1), ["City", "Has_Online_delivery"]]
df_cidades_com_entrega_online = cidades_com_entrega_online.loc[:,["City", "Has_Online_delivery"]].groupby("City").count().sort_values(["Has_Online_delivery"], ascending=False).reset_index()
print(df_cidades_com_entrega_online.head(3))
valor_df_cidades_com_entrega_online = df_cidades_com_entrega_online.iloc[0,1]
print("")
print(f" A cidade que possui mais restaurantes nessas condições é Bhopal com {valor_df_cidades_com_entrega_online} restaurantes.")

        City  Has_Online_delivery
0     Bhopal                   75
1   Vadodara                   74
2  Abu Dhabi                   71

 A cidade que possui mais restaurantes nessas condições é Bhopal com 75 restaurantes.


## Restaurantes

In [170]:
df.head(1)

,Restaurant_ID,Restaurant_Name,Country_Name,City,Address,Locality,Locality_Verbose,Longitude,Latitude,Cuisines,Average_Cost_for_two,Currency,Has_Table_booking,Has_Online_delivery,Is_delivering_now,Price_range,Aggregate_rating,Rating_color,Rating_text,Votes
0,6310675,Mama Lou's Italian Kitchen,Philippines,Las Piñas City,"Block 1, Lot 36, Tropical Avenue Corner Tropic...",BF International,"BF International, Las Piñas City",121.009787,14.447615,Italian,1100,Botswana Pula(P),1,0,0,expensive,4.6,darkgreen,Excellent,619


#### 1. Qual o nome do restaurante que possui a maior quantidade de avaliações?

In [226]:
restaurante_maior_quant_avaliacoes = df.loc[:, ["Restaurant_Name", "Votes"]].groupby("Restaurant_Name").sum().sort_values(["Votes"],ascending=False).reset_index()
print(restaurante_maior_quant_avaliacoes.head(3))
valor_restaurante_maior_quant_avaliacoes = restaurante_maior_quant_avaliacoes.iloc[0,1]
print("")
print(f"O restaurante com maior quantidade de avaliações é o Domino's Pizza com {valor_restaurante_maior_quant_avaliacoes} avaliações.")

             Restaurant_Name  Votes
0             Domino's Pizza  59749
1  AB's - Absolute Barbecues  58875
2            Barbeque Nation  53200

O restaurante com maior quantidade de avaliações é o Domino's Pizza com 59749 avaliações.


#### 2. Qual o nome do restaurante com a maior nota média?

In [227]:
restaurante_maior_nota_media = df.loc[:,["Restaurant_Name", "Aggregate_rating"]].groupby("Restaurant_Name").max().sort_values(["Aggregate_rating"], ascending=False).reset_index()
print(restaurante_maior_nota_media.head(20))
valor_restaurante_maior_nota_media = restaurante_maior_nota_media.iloc[0,1]
print("")
print(f" O restaurante com a maior nota média é o Cut By Wolfgang Puck com nota de {valor_restaurante_maior_nota_media}.")

               Restaurant_Name  Aggregate_rating
0         Cut By Wolfgang Puck               4.9
1  Byg Brewski Brewing Company               4.9
2              Texas de Brazil               4.9

 O restaurante com a maior nota média é o Cut By Wolfgang Puck com nota de 4.9.


In [22]:
df.head(20)

,Restaurant_ID,Restaurant_Name,Country_Name,City,Address,Locality,Locality_Verbose,Longitude,Latitude,Cuisines,Average_Cost_for_two,Currency,Has_Table_booking,Has_Online_delivery,Is_delivering_now,Price_range,Aggregate_rating,Rating_color,Rating_text,Votes
0,6310675,Mama Lou's Italian Kitchen,Philippines,Las Piñas City,"Block 1, Lot 36, Tropical Avenue Corner Tropic...",BF International,"BF International, Las Piñas City",121.009787,14.447615,Italian,1100,Botswana Pula(P),1,0,0,expensive,4.6,darkgreen,Excellent,619
2,6314542,Blackbird,Philippines,Makati City,"Nielson Tower, Ayala Triangle Gardens, Salcedo...","Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...",121.024562,14.556042,European,3100,Botswana Pula(P),0,0,0,gourmet,4.7,darkgreen,Excellent,469
3,6301293,Banapple,Philippines,Makati City,"Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...",121.023171,14.556196,Filipino,800,Botswana Pula(P),0,0,0,expensive,4.4,green,Very Good,867
4,6315689,Bad Bird,Philippines,Makati City,"Hole In The Wall, Floor 4, Century City Mall, ...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027708,14.565899,American,700,Botswana Pula(P),0,0,0,expensive,4.4,green,Very Good,858
5,6304833,Manam,Philippines,Makati City,"Level 1, Greenbelt 2, Ayala Center, Greenbelt,...","Greenbelt 2, San Lorenzo, Makati City","Greenbelt 2, San Lorenzo, Makati City, Makati ...",121.020380,14.552351,Filipino,700,Botswana Pula(P),0,0,0,expensive,4.7,darkgreen,Excellent,930
6,18409457,Soban K-Town Grill,Philippines,Makati City,"Level 3, Greenbelt 3, Ayala Center, Greenbelt,...","Greenbelt 3, San Lorenzo, Makati City","Greenbelt 3, San Lorenzo, Makati City, Makati ...",121.021388,14.552248,Korean,1300,Botswana Pula(P),0,0,0,expensive,4.7,darkgreen,Excellent,935
7,18607559,Bluesmith Coffee & Kitchen,Philippines,Makati City,"Level 3, Greenbelt 3, Ayala Center, Greenbelt,...","Greenbelt 3, San Lorenzo, Makati City","Greenbelt 3, San Lorenzo, Makati City, Makati ...",121.021374,14.552044,American,700,Botswana Pula(P),0,0,0,expensive,4.0,green,Very Good,340
8,6314001,Motorino Pizzeria Napoletana,Philippines,Makati City,"Level 2, Greenbelt 3, Ayala Center, Greenbelt,...","Greenbelt 3, San Lorenzo, Makati City","Greenbelt 3, San Lorenzo, Makati City, Makati ...",121.021772,14.551875,Pizza,1000,Botswana Pula(P),0,1,1,expensive,4.3,green,Very Good,449
9,18189398,Shi Lin,Philippines,Makati City,"Level 3, Greenbelt 3, Ayala Center, Greenbelt,...","Greenbelt 3, San Lorenzo, Makati City","Greenbelt 3, San Lorenzo, Makati City, Makati ...",121.021653,14.552189,Taiwanese,1000,Botswana Pula(P),0,1,0,expensive,4.1,green,Very Good,201
10,6318945,Nikkei,Philippines,Makati City,"Unit GO3, Frabelle Business Center, 111 Rada S...",Legaspi Village,"Legaspi Village, Makati City",121.018108,14.555784,Japanese,1600,Botswana Pula(P),1,0,0,gourmet,4.6,darkgreen,Excellent,394


#### 3. Qual o nome do restaurante que possui o maior valor de um prato para duas pessoas?

In [228]:
restaurante_maior_valor = df.loc[:,["Restaurant_Name", "Average_Cost_for_two"]].groupby("Restaurant_Name").max().sort_values(["Average_Cost_for_two"], ascending=False).reset_index()
print(restaurante_maior_valor.head(3))
valor_restaurante_maior_valor = restaurante_maior_valor.iloc[0,1]
print("")
print(f" O restaurante com a maior valor para prato para dois é o d'Arry's Verandah Restaurant com nota de {valor_restaurante_maior_valor}.")

                Restaurant_Name  Average_Cost_for_two
0  d'Arry's Verandah Restaurant              25000017
1        The Café - Hotel Mulia               1200000
2          GIA Restaurant & Bar                700000

 O restaurante com a maior valor para prato para dois é o d'Arry's Verandah Restaurant com nota de 25000017.


#### 4. Qual o nome do restaurante de tipo de culinária brasileira que possui a menor média de avaliação?

In [229]:
restaurantes_brasil = df.loc[(df["Cuisines"] == "Brazilian") & (df["Aggregate_rating"] >= 0.1), ["Restaurant_Name", "Cuisines", "Aggregate_rating"]]
df_menor_avaliacao_restaurantes_brasil = restaurantes_brasil.loc[:, ["Restaurant_Name", "Cuisines", "Aggregate_rating"]].groupby("Restaurant_Name").min().sort_values(["Aggregate_rating"], ascending=True).reset_index()
print(df_menor_avaliacao_restaurantes_brasil.head(3))
valor_df_menor_avaliacao_restaurantes_brasil = df_menor_avaliacao_restaurantes_brasil.iloc[0,2]
print("")
print(f"O restaurante de culinária brasileira que possui a menor média de avaliação é o Café das Estrelas com {valor_df_menor_avaliacao_restaurantes_brasil} de média.")

           Restaurant_Name   Cuisines  Aggregate_rating
0        Café das Estrelas  Brazilian               3.0
1  Confraria Chico Mineiro  Brazilian               3.3
2               Liv Lounge  Brazilian               3.3

O restaurante de culinária brasileira que possui a menor média de avaliação é o Café das Estrelas com 3.0 de média.


#### 5. Qual o nome do restaurante de tipo de culinária brasileira, e que é do Brasil, que possui a maior média de avaliação?

In [230]:
restaurantes_brasil = df.loc[(df["Cuisines"] == "Brazilian") & (df["Aggregate_rating"] >= 0.1), ["Restaurant_ID", "Restaurant_Name", "Cuisines", "Aggregate_rating"]]
df_maior_avaliacao_restaurantes_brasil = restaurantes_brasil.loc[:, ["Restaurant_ID","Restaurant_Name", "Aggregate_rating"]].groupby("Restaurant_Name").max().sort_values(["Aggregate_rating"], ascending=False).reset_index()
print(df_maior_avaliacao_restaurantes_brasil.head(4))
valor_df_maior_avaliacao_restaurantes_brasil = df_maior_avaliacao_restaurantes_brasil.iloc[0,2]
print("")
print(f"O restaurante de culinária brasileira que possui a menor média de avaliação é o Braseiro da Gávea com {valor_df_maior_avaliacao_restaurantes_brasil} de média.")

         Restaurant_Name  Restaurant_ID  Aggregate_rating
0      Braseiro da Gávea        7300955               4.9
1              Aprazível        7302898               4.9
2        Texas de Brazil       18403563               4.9
3  Ola Brasil Restaurant       18667552               4.9

O restaurante de culinária brasileira que possui a menor média de avaliação é o Braseiro da Gávea com 4.9 de média.


#### 6. Os restaurantes que aceitam pedido online são também, na média, os restaurantes que mais possuem avaliações registradas?

In [231]:
restaurantes_com_entrega_online_votes = df.loc[(df["Has_Online_delivery"] == 1), ["Votes"]].sum()
lista_restaurantes_com_entrega_online = list(restaurantes_com_entrega_online_votes)
valor_lista_restaurantes_com_entrega_online_votes = lista_restaurantes_com_entrega_online[0]
restaurantes_com_entrega_online = df.loc[(df["Has_Online_delivery"] == 1),"Restaurant_ID"].count()

##################################
restaurantes_sem_entrega_online_votes = df.loc[(df["Has_Online_delivery"] == 0), ["Votes"]].sum()
lista_restaurantes_sem_entrega_online = list(restaurantes_sem_entrega_online_votes)
valor_lista_restaurantes_sem_entrega_online_votes = lista_restaurantes_sem_entrega_online[0]
restaurantes_sem_entrega_online = df.loc[(df["Has_Online_delivery"] == 0),"Restaurant_ID"].count()

####################################
media_restaurantes_com_entrega_online_votes = round((valor_lista_restaurantes_com_entrega_online_votes) / (restaurantes_com_entrega_online), 2)
print(media_restaurantes_com_entrega_online_votes)

media_restaurantes_sem_entrega_online_votes = round((valor_lista_restaurantes_sem_entrega_online_votes) / (restaurantes_sem_entrega_online), 2)
print(media_restaurantes_sem_entrega_online_votes)

print(f"Sim, os restaurantes com pedido online possuem, na média, mais avaliações ({media_restaurantes_com_entrega_online_votes}) registradas em comparação aos sem pedido online({media_restaurantes_sem_entrega_online_votes}).")




838.82
479.63
Sim, os restaurantes com pedido online possuem, na média, mais avaliações (838.82) registradas em comparação aos sem pedido online(479.63).


#### 7. Os restaurantes que fazem reservas são também, na média, os restaurantes que possuem o maior valor médio de um prato para duas pessoas?

In [232]:
restaurantes_com_reserva = df.loc[(df["Has_Table_booking"] == 1), "Restaurant_ID"].count()
print(restaurantes_com_reserva)
valor_prato_restaurantes_com_reserva = round(df.loc[(df["Has_Table_booking"] == 1), "Average_Cost_for_two"].mean(),2)
print(valor_prato_restaurantes_com_reserva)
###############################

restaurantes_sem_reserva = df.loc[(df["Has_Table_booking"] == 0), "Restaurant_ID"].count()
print(restaurantes_sem_reserva)
valor_prato_restaurantes_sem_reserva = round(df.loc[(df["Has_Table_booking"] == 0), "Average_Cost_for_two"].mean(),2)
print(valor_prato_restaurantes_sem_reserva)

print("")
print(f"Sim, os restaurantes que fazem reservas possuem, na média, maior valor de um prato para duas pessoas ({valor_prato_restaurantes_com_reserva}) em comparação aos sem reservas({valor_prato_restaurantes_sem_reserva}).")


420
69998.42
6507
3489.63

Sim, os restaurantes que fazem reservas possuem, na média, maior valor de um prato para duas pessoas (69998.42) em comparação aos sem reservas(3489.63).


#### 8. Os restaurantes do tipo de culinária japonesa dos Estados Unidos da América possuem um valor médio de prato para duas pessoas maior que as churrascarias americanas (BBQ)?

In [233]:
restaurantes_japones_usa = df.loc[(df["Country_Name"] == "United States of America") & (df["Cuisines"] == "Japanese"), ["Country_Name", "Cuisines", "Average_Cost_for_two"]]
prato_restaurantes_japones_usa = round(restaurantes_japones_usa.loc[:, "Average_Cost_for_two"].mean(),2)
print(prato_restaurantes_japones_usa)

############################

restaurantes_bbq_usa = df.loc[(df["Country_Name"] == "United States of America") & (df["Cuisines"] == "BBQ"), ["Country_Name", "Cuisines", "Average_Cost_for_two"]]
prato_restaurantes_bbq_usa = round(restaurantes_bbq_usa.loc[:, "Average_Cost_for_two"].mean(),2) 
print(prato_restaurantes_bbq_usa)
print("")
print(f"Sim, os restaurantes Japoneses possuem um valor médio (${prato_restaurantes_japones_usa}) maior nessas condições do que os de BBQ(${prato_restaurantes_bbq_usa})")




#BBQ

56.41
39.64

Sim, os restaurantes Japoneses possuem um valor médio ($56.41) maior nessas condições do que os de BBQ($39.64)


## Tipos de Culinária

In [234]:
df["Cuisines"].unique()

array(['Italian', 'European', 'Filipino', 'American', 'Korean', 'Pizza',
       'Taiwanese', 'Japanese', 'Coffee', 'Chinese', 'Seafood',
       'Singaporean', 'Vietnamese', 'Latin American', 'Healthy Food',
       'Cafe', 'Fast Food', 'Brazilian', 'Argentine', 'Arabian', 'Bakery',
       'Tex-Mex', 'Bar Food', 'International', 'French', 'Steak',
       'German', 'Sushi', 'Grill', 'Peruvian', 'North Eastern',
       'Ice Cream', 'Burger', 'Mexican', 'Vegetarian', 'Contemporary',
       'Desserts', 'Juices', 'Beverages', 'Spanish', 'Thai', 'Indian',
       'BBQ', 'Mongolian', 'Portuguese', 'Greek', 'Asian', 'Author',
       'Gourmet Fast Food', 'Lebanese', 'Modern Australian', 'African',
       'Coffee and Tea', 'Australian', 'Middle Eastern', 'Malaysian',
       'Tapas', 'New American', 'Pub Food', 'Southern', 'Diner', 'Donuts',
       'Southwestern', 'Sandwich', 'Irish', 'Mediterranean', 'Cafe Food',
       'Korean BBQ', 'Fusion', 'Canadian', 'Breakfast', 'Cajun',
       'New Mexican',

#### 1. Dos restaurantes que possuem o tipo de culinária italiana, qual o nome do restaurante com a maior média de avaliação?

In [20]:
restaurantes_italianos = df.loc[(df["Cuisines"] == "Italian"), ["Country_Name", "Restaurant_Name", "Restaurant_ID", "Cuisines", "Aggregate_rating"]]
os_maiores_italianos = restaurantes_italianos.sort_values(["Aggregate_rating"], ascending=False).head().reset_index()
desempate_italianos = os_maiores_italianos.sort_values(["Restaurant_ID"], ascending=True).head()
nome_maior_italianos = desempate_italianos.iloc[0,2]
avaliacao_maior_italianos = desempate_italianos.iloc[0,5]
tipo_culinaria = desempate_italianos.iloc[0,4]
print(desempate_italianos)
print(f"O restaurante {tipo_culinaria} com a maior média de avaliação é o Restaurante {nome_maior_italianos} com {avaliacao_maior_italianos} de média.")


   index              Country_Name                 Restaurant_Name  \
3   5408                     India                         Darshan   
1   6305                New Zeland                           Ombra   
0    366                 Australia    Andre's Cucina & Polenta Bar   
2   1280  United States of America  Perricone's Marketplace & Café   
4   4056                     India   Zolocrust - Hotel Clarks Amer   

   Restaurant_ID Cuisines  Aggregate_rating  
3          10021  Italian               4.9  
1        7100171  Italian               4.9  
0       16587684  Italian               4.9  
2       16923457  Italian               4.9  
4       18209498  Italian               4.9  
O restaurante Italian com a maior média de avaliação é o Restaurante Darshan com 4.9 de média.


#### 2. Dos restaurantes que possuem o tipo de culinária italiana, qual o nome do restaurante com a menor média de avaliação?

In [236]:
restaurantes_italianos = df.loc[(df["Cuisines"] == "Italian") & (df["Aggregate_rating"] >= 0.1), ["Restaurant_Name", "Restaurant_ID", "Cuisines", "Aggregate_rating"]]
os_menores_italianos = restaurantes_italianos.sort_values(["Aggregate_rating"], ascending=True).head(3).reset_index()
print(os_menores_italianos)
nome_menor_italiano = os_menores_italianos.iloc[0, 1]
avaliacao_menor_italiano = os_menores_italianos.iloc[0,4]
print("")
print(f"O restaurante italiano com a menor média de avaliação é o Restaurante {nome_menor_italiano} com {avaliacao_menor_italiano} de média.")


   index Restaurant_Name  Restaurant_ID Cuisines  Aggregate_rating
0   1587         Siena's       16596386  Italian               2.3
1   2137     RPM Italian       18373064  Italian               2.6
2   5354        Satsanga        3700020  Italian               2.8

O restaurante italiano com a menor média de avaliação é o Restaurante Siena's com 2.3 de média.


#### 3. Dos restaurantes que possuem o tipo de culinária americana, qual o nome do restaurante com a maior média de avaliação?

In [22]:
restaurantes_americanos = df.loc[(df["Cuisines"] == "American") & (df["Aggregate_rating"] >= 0.1), ["Restaurant_Name", "Restaurant_ID", "Cuisines", "Aggregate_rating"]]
os_maiores_americanos = restaurantes_americanos.sort_values(["Aggregate_rating"], ascending=False).head(26).reset_index()
desempate_americanos = os_maiores_americanos.sort_values(["Restaurant_ID"], ascending=True)
nome_maior_americano = desempate_americanos.iloc[0,1]
avaliacao_maior_americano = desempate_americanos.iloc[0,4]
print(f"O restaurante americano com a maior média de avaliação é o Restaurante {nome_maior_americano} com {avaliacao_maior_americano} de média.")

O restaurante americano com a maior média de avaliação é o Restaurante Burger & Lobster com 4.9 de média.


#### 4. Dos restaurantes que possuem o tipo de culinária americana, qual o nome do restaurante com a menor média de avaliação?

In [238]:
restaurantes_americanos = df.loc[(df["Cuisines"] == "American") & (df["Aggregate_rating"] >= 0.1), ["Restaurant_Name", "Restaurant_ID", "Cuisines", "Aggregate_rating"]]
os_menores_americanos = restaurantes_americanos.sort_values(["Aggregate_rating"], ascending=True).head().reset_index()
nome_menor_americano = os_menores_americanos.iloc[0,1]
avaliacao_menor_americano = os_menores_americanos.iloc[0,4]
print(f"O restaurante americano com a menor média de avaliação é o Restaurante {nome_menor_americano} com {avaliacao_menor_americano} de média.")

O restaurante americano com a menor média de avaliação é o Restaurante Guy Fieri's Kitchen & Bar com 2.2 de média.


#### 5. Dos restaurantes que possuem o tipo de culinária árabe, qual o nome do restaurante com a maior média de avaliação?

In [23]:
restaurantes_arabes = df.loc[(df["Cuisines"] == "Arabian") & (df["Aggregate_rating"] >= 0.1), ["Restaurant_Name", "Restaurant_ID", "Cuisines", "Aggregate_rating"]]
os_maiores_arabes = restaurantes_arabes.sort_values(["Aggregate_rating"], ascending=False).head().reset_index()
nome_maior_arabe = os_maiores_arabes.iloc[0,1]
avaliacao_maior_arabe = os_maiores_arabes.iloc[0,4]
print(f"O restaurante árabe com a maior média de avaliação é o Restaurante {nome_maior_arabe} com {avaliacao_maior_arabe} de média.")
print(avaliacao_maior_arabe)

O restaurante árabe com a maior média de avaliação é o Restaurante Mandi@36 com 4.7 de média.


#### 6. Dos restaurantes que possuem o tipo de culinária árabe, qual o nome do restaurante com a menor média de avaliação?

In [240]:
restaurantes_arabes = df.loc[(df["Cuisines"] == "Arabian") & (df["Aggregate_rating"] >= 0.1), ["Restaurant_Name", "Restaurant_ID", "Cuisines", "Aggregate_rating"]]
os_menores_arabes = restaurantes_arabes.sort_values(["Aggregate_rating"], ascending=True).head().reset_index()
nome_menor_arabe = os_menores_arabes.iloc[0,1]
avaliacao_menor_arabe = os_menores_arabes.iloc[0,4]
print(f"O restaurante árabe com a menor média de avaliação é o Restaurante {nome_menor_arabe} com {avaliacao_menor_arabe} de média.")

O restaurante árabe com a menor média de avaliação é o Restaurante Empório Árabe com 2.7 de média.


#### 7. Dos restaurantes que possuem o tipo de culinária japonesa, qual o nome do restaurante com a maior média de avaliação?

In [241]:
restaurantes_japones = df.loc[(df["Cuisines"] == "Japanese") & (df["Aggregate_rating"] >= 0.1), ["Restaurant_Name", "Restaurant_ID", "Cuisines", "Aggregate_rating"]]
os_maiores_japoneses = restaurantes_japones.sort_values(["Aggregate_rating"], ascending=False).head(17).reset_index()
desempate_japoneses = os_maiores_japoneses.sort_values(["Restaurant_ID"], ascending=True)
nome_maior_japones = desempate_japoneses.iloc[0,1]
avaliacao_maior_japones = desempate_japoneses.iloc[0,4]
print(f"O restaurante japônes com a maior média de avaliação é o Restaurante {nome_maior_japones} com {avaliacao_maior_japones} de média.")

O restaurante japônes com a maior média de avaliação é o Restaurante Sushi Samba com 4.9 de média.


#### 8. Dos restaurantes que possuem o tipo de culinária japonesa, qual o nome do restaurante com a menor média de avaliação?

In [242]:
restaurantes_japones = df.loc[(df["Cuisines"] == "Japanese") & (df["Aggregate_rating"] >= 0.1), ["Restaurant_Name", "Restaurant_ID", "Cuisines", "Aggregate_rating"]]
os_menores_japoneses = restaurantes_japones.sort_values(["Aggregate_rating"], ascending=True).head().reset_index()
nome_menor_japones = os_menores_japoneses.iloc[0,1]
avaliacao_menor_japones = os_menores_japoneses.iloc[0,4]
print(f"O restaurante japônes com a menor média de avaliação é o Restaurante {nome_menor_japones} com {avaliacao_menor_japones} de média.")

O restaurante japônes com a menor média de avaliação é o Restaurante Orienthai com 2.4 de média.


#### 9. Dos restaurantes que possuem o tipo de culinária brasileira, qual o nome do restaurante com a maior média de avaliação?

In [243]:
restaurantes_brasileiros = df.loc[(df["Cuisines"] == "Brazilian") & (df["Aggregate_rating"] >= 0.1), ["Restaurant_Name", "Restaurant_ID", "Cuisines", "Aggregate_rating"]]
os_maiores_brasileiros = restaurantes_brasileiros.sort_values(["Aggregate_rating"], ascending=False).head(4).reset_index()
desempate_brasileiros = os_maiores_brasileiros.sort_values(["Restaurant_ID"], ascending=True)
nome_maior_brasileiro = desempate_brasileiros.iloc[0,1]
avaliacao_maior_brasileiro = desempate_brasileiros.iloc[0,4]
print(f"O restaurante brasileiro com a maior média de avaliação é o Restaurante {nome_maior_brasileiro} com {avaliacao_maior_brasileiro} de média.")

O restaurante brasileiro com a maior média de avaliação é o Restaurante Braseiro da Gávea com 4.9 de média.


#### 10. Dos restaurantes que possuem o tipo de culinária brasileira, qual o nome do restaurante com a menor média de avaliação?

In [244]:
restaurantes_brasileiros = df.loc[(df["Cuisines"] == "Brazilian") & (df["Aggregate_rating"] >= 0.1), ["Restaurant_Name", "Restaurant_ID", "Cuisines", "Aggregate_rating"]]
os_menores_brasileiros = restaurantes_brasileiros.sort_values(["Aggregate_rating"], ascending=True).head(4).reset_index()
nome_menor_brasileiro = os_menores_brasileiros.iloc[0,1]
avaliacao_menor_brasileiro = os_menores_brasileiros.iloc[0,4]
print(f"O restaurante brasileiro com a menor média de avaliação é o Restaurante {nome_menor_brasileiro} com {avaliacao_menor_brasileiro} de média.")

O restaurante brasileiro com a menor média de avaliação é o Restaurante Café das Estrelas com 3.0 de média.


#### 11. Qual o tipo de culinária que possui o maior valor médio de um prato para duas pessoas?

In [245]:
culinaria_maior_prato = df.loc[:, ["Cuisines", "Average_Cost_for_two"]].sort_values(["Average_Cost_for_two"], ascending=False).reset_index()
tipo_culinaria_maior_prato = culinaria_maior_prato.iloc[0,1]
custo_tipo_culinaria_maior_prato = culinaria_maior_prato.iloc[0,2]
print(f"O tipo de culinária com o maior valor nessas condições é o {tipo_culinaria_maior_prato} com custo médio de {custo_tipo_culinaria_maior_prato}.")

O tipo de culinária com o maior valor nessas condições é o Modern Australian com custo médio de 25000017.


#### 12. Qual o tipo de culinária que possui a maior nota média?

In [38]:
culinaria_maior_media = round(
            df.loc[
                (df["Votes"] >= 600),
                [
                    "Cuisines",
                    "Aggregate_rating",
                ],
            ]
            .groupby("Cuisines")
            .mean()
            .sort_values(["Aggregate_rating"], ascending=False)
            .reset_index(),
            2,
        )
culinaria_maior_media.head(20)


,Cuisines,Aggregate_rating
0,Ramen,4.90
1,Others,4.90
2,British,4.80
3,Caribbean,4.80
4,Egyptian,4.80
5,New American,4.75
6,Sunda,4.75
7,Southern,4.74
8,Patisserie,4.73
9,Sushi,4.73


In [37]:
culinaria_pior_media = round(
            df.loc[
                (df["Votes"] >= 100),
                [
                    "Cuisines",
                    "Aggregate_rating",
                ],
            ]
            .groupby("Cuisines")
            .mean()
            .sort_values(["Aggregate_rating"], ascending=True)
            .reset_index(),
            2,
        )
culinaria_pior_media.head(30)

,Cuisines,Aggregate_rating
0,Australian,3.80
1,Mongolian,3.80
2,Awadhi,3.90
3,Asian Fusion,3.90
4,Naga,3.90
5,Yum Cha,3.90
6,Biryani,3.99
7,Rajasthani,4.00
8,Kokoreç,4.00
9,Home-made,4.00


In [30]:
df["Votes"].mean()

605.5334199509167

#### 13. Qual o tipo de culinária que possui mais restaurantes que aceitam pedidos online e fazem entregas?

In [247]:
culinaria_mais_online_entrega = df.loc[(df["Has_Online_delivery"] == 1) & (df["Is_delivering_now"] == 1), ["Cuisines","Is_delivering_now"]].groupby("Cuisines").count().sort_values(["Is_delivering_now"], ascending=False).reset_index()
print(culinaria_mais_online_entrega.head())
nome_culinaria_mais_online_entrega = culinaria_mais_online_entrega.iloc[0,0]
valor_culinaria_mais_online_entrega = culinaria_mais_online_entrega.iloc[0,1]
print("")
print(f"O tipo de culinária que possui mais restaurantes nessas condições é o {nome_culinaria_mais_online_entrega} com {valor_culinaria_mais_online_entrega} restaurantes")


       Cuisines  Is_delivering_now
0  North Indian                317
1          Cafe                131
2  South Indian                 79
3        Burger                 76
4     Fast Food                 72

O tipo de culinária que possui mais restaurantes nessas condições é o North Indian com 317 restaurantes
